In [0]:
import requests
import json
from datetime import datetime

ingest_date = datetime.utcnow().strftime("%Y-%m-%d")

print(f"📅 Ingest date: {ingest_date}")

# ---------------------------------------------
# 2. Config
# ---------------------------------------------
url = "https://api.openbrewerydb.org/v1/breweries"

# ADLS Bronze Raw Path (JSON Lines)
bronze_raw_dir = f"/mnt/datalake/bronze/breweries/date={ingest_date}"
bronze_raw_file = f"{bronze_raw_dir}/raw.json"

print(f"📂 Saving to: {bronze_raw_file}")

# ---------------------------------------------
# 3. Call the API
# ---------------------------------------------
response = requests.get(url, timeout=30)
response.raise_for_status()

data = response.json()  # list of dictionaries

print(f"✅ Downloaded {len(data)} brewery records")

# ---------------------------------------------
# 4. Save raw JSON as JSON Lines (Bronze best practice)
# ---------------------------------------------
dbutils.fs.mkdirs(bronze_raw_dir)  # create folder if not exists

json_lines = "\n".join(json.dumps(row) for row in data)

dbutils.fs.put(
    bronze_raw_file,
    json_lines,
    overwrite=True
)

print("✔ Raw JSON Lines saved (Bronze layer completed!)")


In [0]:
df_raw = spark.read.json("/mnt/datalake/bronze/breweries")
display(df_raw)

In [0]:
%fs
ls /mnt/datalake/bronze/breweries/